# 1. Mô hình embedding

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [2]:
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings_Model = HuggingFaceEmbeddings(
    model_name="huyydangg/DEk21_hcmute_embedding",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27928\2870113787.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_Model = HuggingFaceEmbeddings(


# 2. Kết nối đến Qdrant

In [3]:
from langchain_qdrant import QdrantVectorStore

In [4]:
exist_GD_Collection = QdrantVectorStore.from_existing_collection(
    embedding = embeddings_Model,
    prefer_grpc=True,
    url = "http://localhost:6333/",
    collection_name = "Domain_Law",
	metadata_payload_key="metadata"
)

# 3. Search không kèm Filter

In [5]:
from pyvi import ViTokenizer

In [6]:
user_query = "Trong lĩnh vực Quyền dân sự, Thông tư số 04/2020/TT-BTP quy định như thế nào khi người yêu cầu khai tử không có giấy tờ chứng minh hợp lệ về cái chết của người thân?"

In [7]:
user_query = ViTokenizer.tokenize(user_query)

In [ ]:
# Thực hiện tìm kiếm trên Qdrant với filter `should`
search_results = exist_GD_Collection.similarity_search_with_score(
    query=user_query,
    filter=None,
    k=5
)

In [27]:
def print_Results_MoreKeywords(results):
    for result in results:
        doc = result[0]  # lấy phần tử đầu tiên
        score_Qdrant = result[1]  # lấy phần tử thứ hai
        print("Page_Content:\n", doc.page_content)
        print("Metadata:", doc.metadata['Loai-Van-Ban'])
        print("Stt:",doc.metadata['STT'])
        print("Điều:",doc.metadata["Article"])
        print("Score Qdrant:",score_Qdrant,"\n","-----------------------")

In [28]:
print_Results_MoreKeywords(search_results)

Page_Content:
 Việc đăng_ký khai_tử được thực_hiện theo quy_định tại Mục 7 Chương II Luật hộ_tịch và hướng_dẫn sau đây : 
 1 . Trường_hợp đăng_ký khai_tử cho người chết đã lâu , không có Giấy_báo_tử hoặc giấy_tờ thay_thế Giấy_báo_tử theo quy_định tại khoản 2 Điều 4 Nghị_định số 123 / 2015 / NĐ - CP thì người yêu_cầu đăng_ký khai_tử phải cung_cấp được giấy_tờ , tài_liệu , chứng_cứ do cơ_quan , tổ_chức có thẩm_quyền cấp hoặc xác_nhận hợp_lệ chứng_minh sự_kiện chết . 2 . Trường_hợp người yêu_cầu đăng_ký khai_tử không có giấy_tờ , tài_liệu , chứng_cứ chứng_minh theo quy_định tại khoản 1 Điều này hoặc giấy_tờ , tài_liệu , chứng_cứ không hợp_lệ , không bảo_đảm giá_trị chứng_minh thì cơ_quan đăng_ký hộ_tịch từ_chối đăng_ký khai_tử .
Metadata: Thông tư
Stt: 2
Điều: Điều 13. Đăng ký khai tử
Score Qdrant: 0.6974231600761414 
 -----------------------
Page_Content:
 1 . Người có trách_nhiệm đi đăng_ký khai_tử nộp tờ khai theo mẫu quy_định và Giấy_báo_tử hoặc giấy_tờ khác thay giấy_báo_tử cho cơ_qu

# 4. Search có filter

In [25]:
import json

metadata_json_str = '{"Loai-Van-Ban": "Nghị định", ' \
'"Noi-Ban-Hanh": "Chính phủ", ' \
'"So-Hieu": "127/2018/NĐ-CP", ' \
'"Ngay-Ban-Hanh": ["21/09/2018"], ' \
'"LinhVuc-Nganh": ["Giáo dục"]}'

# Chuyển chuỗi JSON thành dictionary
metadata_dict = json.loads(metadata_json_str)

# In ra để kiểm tra
print(metadata_dict)
print(type(metadata_dict))


{'Loai-Van-Ban': 'Nghị định', 'Noi-Ban-Hanh': 'Chính phủ', 'So-Hieu': '127/2018/NĐ-CP', 'Ngay-Ban-Hanh': ['21/09/2018'], 'LinhVuc-Nganh': ['Giáo dục']}
<class 'dict'>


In [26]:
# Thực hiện tìm kiếm trên Qdrant với filter `should`
search_results = exist_GD_Collection.similarity_search_with_score(
    query="",
    filter=metadata_dict,
    k=5
)

ValueError: Protocol message Filter has no "Loai-Van-Ban" field.